## Importing Required libraries

In [18]:
import nltk
# nltk.download('stopwords')
import spacy
# spacy.load("en_core_web_sm")
from pyresparser import ResumeParser
from resume_parser import resumeparse
import os
import docx2txt
import requests
import pandas as pd
from pdfminer.high_level import extract_text
# WOERKING COCDE DONT EDIt
import sys
import textract, PyPDF2, glob
import nltk
import pandas as pd
from names_dataset import NameDataset
nd = NameDataset()
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
# nltk.download('stopwords')
import os
#java_path = r"C:\Program Files\Java\jdk-17.0.2\bin\java.exe"
# os.environ['JAVAHOME'] = java_path
import re
import os
import subprocess
from pdfminer.high_level import extract_text
from nltk.corpus import stopwords
import string
import nltk
from nltk.tag.stanford import StanfordNERTagger
import pdfplumber
from nltk.tag import pos_tag
#files = glob.glob(r"C:\Users\rtd91\Data\Resume.pdf")

## Initializing model object and some variables

In [ ]:

PATH_TO_JAR = "C:\\Users\rtd91\Data\stanford-ner.jar"
PATH_TO_MODEL = "C:\\Users\rtd91\Data\english.all.3class.distsim.crf.ser.gz"
stopwords = set(stopwords.words('english'))
files1 = glob.glob(r"C:\Users\rtd91\Data\resume_samples\*")

tagger = StanfordNERTagger(model_filename=r"C:\Users\rtd91\Data\english.all.3class.distsim.crf.ser.gz",path_to_jar=r"C:\Users\rtd91\Data\stanford-ner.jar", encoding='utf-8')

us_names = nd.get_top_names(n=1000, gender='Male', country_alpha2='US')['US']['M']
indian_last_names = ["Acharya", "Agarwal", "Khatri", "Ahuja", "Anand", "Laghari", "Patel",

"Reddy", "Bakshi", "Anthony", "Babu", "Arya", "Balakrishnan", "Banerjee", "Burman", "Bhatt", "Basu", "Bedi", "Varma", "Dara", "Dalal", "Chowdhury",
"Chabra", "Chadha", "Chakrabarti","Chawla","Ahluwalia", "Amin", "Apte", "Datta", "Deol", "Deshpande", "Dewan", "Lal", "Kohli", "Mangal", "Malhotra", "Jha",
"Joshi","Kapadia", "Iyer", "Jain", "Khanna", "Grover", "Kaur", "Kashyap", "Gokhale", "Ghosh", "Garg", "Dhar", "Gandhi", "Ganguly", "Gupta", "Das", "Chopra", "Dhawan",
"Dixit", "Dubey", "Haldar", "Kapoor", "Khurana", "Kulkarni", "Madan", "Bajwa", "Bhasin", "Chandra", "Chauhan", "Deshmukh", "Dayal", "Dhillon", "Goswami", "Goel", "Mallick",
"Mahajan", "Kumar", "Mani",  "Gill", "Mannan", "Biswas", "Batra", "Bawa", "Mehta", "Mukherjee", "Saxena", "Zacharia", "Shah", "Ray", "Rao", "Purohit", "Parekh", "Thakur", "Singh", "Sharma", "Seth", "Sachdev", "Ranganathan", "Puri", "Pandey", "Naidu", "Modi"]

chinese_last_names = ["Li", "Wang", "Zhang", "Liu", "Chen", "Yang", "Zhao", "Huang", "Zhou",

"Wu", "Xu", "Sun", "Hu", "Zhu", "Gao", "Lin", "He", "Guo", "Ma", "Luo", "Liang",

"Song", "Zheng", "Xie", "Han", "Tang", "Feng", "Yu", "Dong", "Xiao", "Cheng",

"Cao", "Yuan", "Deng", "Xu", "Fu", "Shen", "Zeng", "Peng", "Lu", "Su", "Lu", "Jiang", "Cai", "Jia", "Ding", "Wei", "Xue", "Ye", "Yan", 

"Yu", "Pan", "Du", "Dai", "Xia", "Zhong", "Wang", "Tian", "Ren", "Jiang", "Fan", "Fang", "Shi", "Yao", "Tan", "Sheng", "Zou", "Xiong", "Jin", "Lu", "Hao", "Kong", "Bai", "Cui",

"Kang", "Mao", "Qio", "Qin", "Jiang", "Shu", "Shi", "Gu", "Hou", "Shao", "Meng", "Long", "Wan", "Duan", "Zhang", "Qian", "Tang", "Yin", "Li", "Yi", "Chang", "Wu", 
    
"Qiao", "He", "Lao", "Gong", "Wen"]

chinese_last_names = [chinese_last_name.lower() for chinese_last_name in chinese_last_names]
indian_last_names = [indian_last_name.lower() for indian_last_name in indian_last_names]
RESERVED_WORDS = [
    'school',
    'college',
    'univers',
    'academy',
    'faculty',
    'institute',
    'faculdades',
    'Schola',
    'schule',
    'lise',
    'lyceum',
    'lycee',
    'polytechnic',
    'kolej',
    'ünivers',
    'okul',
    'University'
]


## Combined function to calculate all the details about the  applicant

In [28]:
## Tokenize the text from the .pdf file
def tokenize(i, person=[], education=[], graduation_year=[], phone_nbr=[], emails=[]):
    ###Extract text from files
    try:
        with pdfplumber.open(files1[i]) as pdf:
            first_page = pdf.pages[0]
            txt1 = first_page.extract_text()
    except:
        txt1=""
        person.append(files1[i])
        education.append("Err")
        graduation_year.append(extract_graduation_date("Err"))
        phone_nbr.append(extract_phone_number("Err"))
        emails.append(extract_emails("Err"))
        i+=1
        tokenize(i)
    txt = extract_text(files1[i], codec='utf-8')
    #print(txt)
    words = nltk.word_tokenize(txt) 
    return words,i,txt,txt1,person,education,graduation_year,phone_nbr,emails

## Doing some preprocessing steps and appending the details of the applicants
def preprocessing(i=0, person=[],education=[],graduation_year=[], phone_nbr=[],emails=[]):
    ##Here i is for iterating over the files and doing the process for each file
    flag_success = 1 # Trying to check if try was successful for appending person record
    words,i,txt,txt1,person,education,graduation_year,phone_nbr,emails = tokenize(i)
    #print(txt)
    ##removing stopwords and punctuations
    simple_strings = [word for word in words if word not in stopwords if word not in string.punctuation]
    res = [re.sub(r'[^\w\s]', '', word) for word in simple_strings]
    res1 = [re.sub(r'^abc(.*?)=[A-Z0-9]+(.*)', r'\1\2', word) for word in res]
    res2 = [str(res) for res in res1]
    res3 = [re.sub(r'/^[A-Za-z]+$/', '', res) for res in res2]
    
    ##removing some ascii character
    res3 = [res.replace("ï","i") if "ï" in res else res for res in res3]
    
    ##removing numbers
    res3 = [re.sub('\d', '', res) for res in res3]
    
    ##trying to remove acii characters
    res3 = [res.encode('ascii',"ignore").decode() for res in res3]
    
    ##using the tagger object of StanfordNER for tagging the entities of the words 
    tagged = tagger.tag(res3)
    
    ## appending the names extracted from name_person
    ##Since 2 files are of weird format ( 1 is image file converted into .pdf)
    try:
        person.append(name_person(tagged,words,res3))
    except:
        flag_success = 0
        #person.append(files1[i])
        pass
    #tagged_list.append(tagged) 
    #flag_success = 1
    if flag_success == 1:
        education.append(extract_education(txt))
        graduation_year.append(extract_graduation_date(txt1))
        phone_nbr.append(extract_phone_number(txt))
        emails.append(extract_emails(txt))
    else:
        flag_success == 1
    if i+1 < len(files1):
        i+=1
        preprocessing(i, person)
    
    #len(person), len(education), len(graduation_year), len(phone_nbr), len(emails)#
    return pd.DataFrame({'name':person,'education':education, 'graduation_year':graduation_year, 'Contact':phone_nbr, 'email':emails})



def name_person(tagged, words,res3):
    #tagged_list = preprocessing(0,[],[])
    #print(tagged_list)
    person = []
    temp_person = []
    nltk_tagged = pos_tag(res3[:10])
    
    for k in range(10):
        if k<=9:
            if nltk_tagged[k][1] == 'NNP' and nltk_tagged[k+1][1] == 'NNP':
                nltk_name = nltk_tagged[k][0] +' '+ nltk_tagged[k+1][0] 
            return nltk_name
        print()pyr
        if (res3[k].lower() in indian_last_names) or (res3[k].lower() in chinese_last_names):
            j = k-1
            return res3[j]+" " +res3[k]#+"(Extracted using 1st approach)"
    for tuple_ele in tagged:
        if "PERSON" in tuple_ele[1]:
            temp_person.append(tuple_ele[0])
    ## loop through the original words so we can extract from the first words and get the first PERSON
    for word in words:
        if word in temp_person:
            return word

def extract_education(txt):
    edu=set()
    p = re.compile('(EDUCATION)?\n?(.*?),\s+(.*?),(.*?)') 
    for m in re.finditer(p,txt):
        try:
            if any(x in m.group(1) for x in RESERVED_WORDS):
                edu.append(m.group(1))
        except:
            pass
        for word in RESERVED_WORDS:
            if word in m.group(2):
                edu.add(m.group(2))
    return edu

def extract_graduation_date(txt1):
    dates=["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
    #/^(?=^abc)(?=.*xyz$)(?=.*123)(?=^(?:(?!456).)*$).*$/
    # Working to extract Months :x="(?=("+'|'.join(dates)+r"))"
    x="(?is)education.*?(\d{4})"
    # Working to extract year after education: x="(?is)education.*?(\d{4})"
    if len(re.findall(x,txt1))==0:
        return None
    return max(re.findall(x,txt1))
    ## for dates
    #for dt in dates:
    #    if dt in txt:
    #        return dt

def extract_phone_number(txt):
    PHONE_REG = re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]')
    phone = re.findall(PHONE_REG, txt)

    if phone:
        number = ''.join(phone[0])

        if txt.find(number) >= 0 and len(number) < 16:
            return number
    return None

def extract_emails(txt):
    EMAIL_REG = re.compile(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')
    return re.findall(EMAIL_REG, txt)

StanfordNER = preprocessing()
StanfordNER.head()

,name,education,graduation_year,Contact,email
0,Karthik Ramanarayana,{University of Maryland},2021,(410)-292-1151,[karthikr2194@gmail.com]
1,HARSH PUNDIR,{},2020,None,[hpundir@umd.edu]
2,TIRTH PATEL,"{The University of Texas at Dallas, Nirma Univ...",2021,469-370-9437,[tirth2410@gmail.com]
3,Akanksha Bapna,{},2021,None,[]
4,Mythri Partha,{University of Houston},2015,(281) 725-7080,[mythripartha8@gmail.com]


## Evaluating results using fuzzy logic

In [31]:
ground_truth_tail = ground_truth
# !pip install fuzzywuzzy
# !pip install python-Levenshtein
from fuzzywuzzy import fuzz
def metrics(parse_col,ground_truth_col):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    list1 = ground_truth_col.to_list()
    list2 = parse_col.to_list()

    
    for i in range(len(list1)):
#         print(fuzz.ratio(str.lower(list2[i]), list1[i]))
        if list2[i]==None:
            fp+=1
            continue
        try:
           
            if fuzz.ratio(str.lower(list2[i]), str.lower(list1[i]))>30:
                print(list1[i],list2[i])
                tp+=1
            else:
                print(f'NO MATCH {list1[i]}, {list2[i]}. Score = {fuzz.ratio(str.lower(list2[i]), list1[i])}')
                fp+=1
        except:
            
            fp+=1
    print(i)
    i+=1
    return tp,fp

metric_list = []
StanfordNER["mobile"] = 0
StanfordNER["degree"] = ""
### Scoring for pyres parser
name = metrics(StanfordNER['name'],ground_truth_tail['name'])
# email = metrics(StanfordNER['email'],ground_truth_tail['email'])
# mobile = metrics(StanfordNER['Contact'],ground_truth_tail['Contact'])
# university = metrics(StanfordNER['education'],ground_truth_tail['education'])
# degree = metrics(StanfordNER['degree'],ground_truth_tail['Major'])

# metric_list.append({'name':name,'email':email,'mobile':mobile,'university':university,'degree':degree})
name
#print(name,email,mobile,university,degree)

Karthik Ramanarayana Karthik Ramanarayana
HARSH PUNDIR HARSH PUNDIR
TIRTH PATEL TIRTH PATEL
Akanksha Bapna Akanksha Bapna
Mythri Partha Mythri Partha
Qizhe Wang Qizhe Wang
RACHAN BHOOSHI RACHAN VAMSI
RAGHUVEERA MADIREDDY RAGHUVEERA KARTHIK
SAIDA MUKTAR SAIDA MUKTAR
Akash Patel Allentown PA
AKSHARA ARCOT AKSHARA ARCOT
Alexandra Manetas Alexandra Manetas
NO MATCH Pranav Gulghane, MOHIT MANJARIA. Score = 28
NO MATCH Wanting Lu, Yuchen Xie. Score = 10
NO MATCH Manaswini Nagaraj, Indupriya Kompi. Score = 25
MOHIT MANJARIA James Renier
NO MATCH Yuchen Xie, ANYA PATEL. Score = 30
NO MATCH Dimple Mehra, Ryan Martin. Score = 17
NO MATCH Indupriya Kompi, Mayank Dubey. Score = 22
NO MATCH James Domingo, ABHINAY NAGULAPALLI. Score = 25
ANYA PATEL Noah Mattheis
NO MATCH Ryan Goodwin, Abhilash Rajaram. Score = 14
Mayank Dubey Adam Dinser
ABHINAY NAGULAPALLI AJAY PRASATH
NO MATCH Noah Mattheis, Akshay Kajale. Score = 15
Abhilash Rajaram ALEXANDER FRIEND
NO MATCH Adam Dinser, Amar Nath. Score = 30
ADE

C:\Users\rtd91\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


IndexError: list index out of range